In [1]:
import ev3_dc as ev3
from thread_task import Periodic, Task, Repeated, STATE_STARTED, STATE_FINISHED, STATE_STOPPED, STATE_TO_STOP
import cv2
import numpy as np
import cv2.aruco as aruco
import time

In [3]:
camera_matrix = np.array([[635.93, 0.0, 444.27], [0.0, 634.263, 256.835],[0.0, 0.0, 1.0]])
distortion_coefficient =  np.array([-0.372403, 0.157163, -0.00424357, -0.00231644, -0.0522724])
translation_matrix = np.array([[1 ,0, 0],
                               [0, np.cos(120), -np.sin(120)],
                               [0, np.sin(120), np.cos(120)]])

In [4]:
def getDistance(camera_matrix,distortion_coefficient, cap, camera_height):
    tvecs = []
    ret, frame = cap.read()
    print(type(frame))
    # frame = np.flipud(frame)
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  

    aruco_dict = aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)  
    parameters = aruco.DetectorParameters_create()  
    corners, ids, rejected_img_points = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)   
    
    min_dist = np.inf

    if ids is not None:
        for j in range(0, len(ids)): 
            rvec, tvec, _ = aruco.estimatePoseSingleMarkers(corners[j], 0.048, camera_matrix, distortion_coefficient)
            hypothenuse = np.linalg.norm(tvec)
            distance = np.sqrt(hypothenuse**2 - camera_height**2)
            if distance < min_dist:
                min_dist = distance
                min_id = ids[j]

        return distance, min_id
    return np.inf, None

In [5]:
camera = cv2.VideoCapture(1)


In [6]:
camera.release()
cv2.destroyAllWindows()


In [6]:
with ev3.TwoWheelVehicle(
    0.0210,  # radius_wheel
    0.1350,  # tread
    protocol=ev3.USB
) as vehicle:
    turn_180 = ( 
        vehicle.drive_turn(180,0)
    )

    drive_10 = (
        vehicle.drive_straight(0.1)
    )

    def checkForMarkers():
        camera_height = 0.285
        curr_state = straight.state
        min_dist, min_id = getDistance(camera_matrix,distortion_coefficient, camera, camera_height)
        pos = vehicle.position
        print("Current minimal distance: {} with id {}".format(min_dist, min_id))
        # print("Current position: {}".format(pos))

        if curr_state == STATE_STOPPED:
            return True

        if (curr_state == STATE_STARTED and min_dist < 0.4):
            print("Current minimal distance: {} with id {}".format(min_dist, min_id))
            print("WAY TOO CLOSE MAN!")
            straight.stop()
            print("STOPPING STATE: {}".format(straight.state))
            return True

        # elif (curr_state == STATE_STOPPED and (min_dist is None or min_dist >= 0.4)):
        #     straight.cont()

        return False  

    straight = (
        Repeated(
            vehicle.drive_straight(0.01) +
            Task(checkForMarkers)
        )
    )

    # drive_and_check = (
    #     Task(straight.start) +
    #     Periodic(
    #         1,  # interval
    #         checkForMarkers
    #     ) +
    #     Task(straight.stop) + 
    #     Task(turn_180.start)
    # ).start(thread=False)

    parcour = Repeated(
            Task(straight) + 
            Task(turn_180)
        ).start(thread=False)

    # Task(parcour.start).start(thread=False)


    
    

Current minimal distance: 0.6231442638162991 with id [27]
Current minimal distance: 0.7075323655487263 with id [27]
Current minimal distance: 0.6231442638162991 with id [27]
Current minimal distance: 0.7598469940050975 with id [27]
Current minimal distance: 0.639102119308316 with id [24]
Current minimal distance: 0.615456643520195 with id [24]
Current minimal distance: 0.5994059680954508 with id [27]
Current minimal distance: 0.6578758798511467 with id [27]
Current minimal distance: 0.5830088755104443 with id [24]
Current minimal distance: 0.707216802170079 with id [24]
Current minimal distance: 0.5472594752358635 with id [27]
Current minimal distance: 0.5474973982274732 with id [30]
Current minimal distance: 0.5310390883187515 with id [27]
Current minimal distance: 0.518813730231598 with id [30]
Current minimal distance: 0.507735034322274 with id [27]
Current minimal distance: 0.4901100547985282 with id [27]
Current minimal distance: 0.4776394982263571 with id [27]
Current minimal dis

KeyboardInterrupt: 